# TP 4 - Analyse d'images et de vidéos

Dans ce TP, nous allons analyser et caractériser des images et vidéos.

Ce TP est **noté**. Vous pouvez rendre votre code en m'envoyant le fichier Jupyter (format `ipynb`) par mail à `florent.grelard [at] u-bordeaux.fr`.

Vous répondrez aux questions "ouvertes" ("Que constatez-vous...?", "Pourquoi...?", etc.) dans le TP par des commentaires dans votre code Python. Toutes tentatives et justifications pertinentes seront valorisées.

Vous pouvez vous aider de toutes les fonctions numpy/skimage existantes. Si vous êtes bloqués à une question, vous pouvez utiliser les fonctions de ces bibliothèques pour passer aux questions suivantes.

## 1. Détection d'objets

Dans cet exercice, nous cherchons à détecter et isoler des objets. 

Nous allons extraire les **composantes connexes** de l'image afin d'extraire des mesures indépendamment pour chaque objet. Nous rappelons qu'une composante connexe est obtenue par **étiquetage** des pixels (*labelling* en anglais). Le principe est d'effectuer un **parcours en largeur** à partir d'un pixel objet (=255). Chaque pixel objet dans l'image est parcouru jusqu'à ce qu'ils soient tous étiquetés.

Deux solutions pour le parcours en largeur:
* Utilisation de la récursivité
* Utilisation d'une pile

<table>
    <tr>
    <td>
        <img src="data/obj_cc.png" width="700" /> 
        <figcaption style="text-align:center;"> De gauche à droite: (a) Image originale, (b) Image de labels (chaque couleur correspond à un label différent) </figcaption>
    </td>
    </tr>
</table>

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `eight_neighbours` qui extrait les coordonnées des 8-voisins d'un pixel.
2. Compléter les fonctions `assign_label`, qui effectue le parcours en largeur et étiquète à partir d'**un pixel**, et  `connected_components`, qui étiquète l'**ensemble de l'image**. À l'issue du calcul, chaque composante connexe doit avoir un label différent.
3. Les ballons possèdent un trou avec des pixels noirs. Quelle solution vue en cours pourrait être utilisée pour le combler?
4. Une autre approche est d'utiliser `assign_label` en partant d'un pixel localisé dans un trou. Compléter la fonction `fill_holes`. Les coordonnées des pixels dans chaque trou sont données dans la variable `coords`. Appeler la fonction `fill_holes` **avant** d'effectuer l'étiquetage.
5. (Facultatif) En cas de doute ou de difficulté, on pourra utiliser les fonctions `skimage.measure.label` (composantes connexes) et `scipy.ndimage.binary_fill_holes`.

In [223]:
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage import morphology
from skimage import draw
from skimage import transform

def eight_neighbours(image, x, y):
    """
    Coordonnées (x,y) pour le huit-voisinage
    autour de x, y

    Returns
    -------
    region: list
        Coordonnées (x, y) des 8 voisins du pixel
        Longueur = 8
    """
    L = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i != 0 or j!=0:
                new_x = np.clip(x+i, 0, image.shape[0]-1)
                new_y = np.clip(y+j, 0, image.shape[1]-1)
                L.append(np.array([new_x, new_y]))
    return L


def assign_label(image, labelled_image, current_label, x, y):
    """
    Parcours en largeur et etiquetage des
    pixels objets

    Parameters
    ----------
    image: np.ndarray
        image
    labelled_image: np.ndarray
        image de labels
    current_label: int
        label courant
    x: int
        coordonnée x
    y: int
        coordonnée y

    """
    if labelled_image[x, y] != 0 or image[x, y] == 0:
        return
    labelled_image[x, y] = current_label
    neighborhood = eight_neighbours(labelled_image, x, y)
    for n in neighborhood:
        assign_label(image, labelled_image, current_label, n[0], n[1])


def connected_components(image):
    """
    Extrait l'ensemble des composantes
    connexes dans une image

    Returns
    -------
    labelled_image: np.ndarray
        image de labels
    """
    labelled_image = np.zeros_like(image)
    indices_nonzero = np.argwhere(image != 0)
    current_label = 1
    for x,y in indices_nonzero:
        if labelled_image[x, y] != 0:
            continue
        assign_label(image, labelled_image, current_label, x, y)
        current_label += 1
    return labelled_image


def fill_holes(image, coords):
    """
    Comble les trous dans une image
    à partir des coordonnées passées en paramètres

    Parameters
    ----------

    """
    inverted_image = (255 - image).astype(np.uint8)
    filled_image = image.copy()
    for x,y in coords:
        assign_label(inverted_image, filled_image, 255, x, y)
    return filled_image



img_birds_rgb = io.imread("data/birds_sky.jpg")
img_birds = (color.rgb2gray(img_birds_rgb)*255).astype(np.uint8)

img_threshold = np.where(img_birds < 30, 255, 0)

coords = np.array([[254, 274], [206, 324], [106, 402]])
img_filled = fill_holes(img_threshold, coords)

img_labels = connected_components(img_filled)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img_birds_rgb)
ax[1].imshow(img_labels)
[axi.set_axis_off() for axi in ax]

[None, None]

## 2. Caractérisation quantitative

À ce stade, nous avons extrait et séparé les différents objets. Deux types d'objets sont toutefois présents dans l'image: des oiseaux et des ballons. Nous cherchons des mesures permettant de les discriminer.

<table>
    <tr>
    <td>
        <img src="data/obj_mesure.png" width="700" /> 
        <figcaption style="text-align:center;"> Classification des objets par des mesures: (a) Image des ballons, (b) Image des oiseaux </figcaption>
    </td>
    </tr>
</table>

Voici quelques mesures qui peuvent être pertinentes:
* L'aire : nombre de pixels dans la composante

* Le périmètre : peut être (mal) estimé par le nombre de pixels dans le contour de la composante

* La compacité: $C = \dfrac{4 \pi \text{Aire}}{\text{Périmètre}^2}$

* La circularité $C_{box} = \dfrac{4 \pi \text{Aire}}{\text{Périmètre boite}^2}$

* L'élongation : $E = \dfrac{\text{Longueur boite}}{\text{Largeur boite}}$.

$\text{boite}$ : boîte englobante = le rectangle qui contient l'ensemble de l'objet.

**Exercice**: Compléter le code suivant:
1. Pourquoi le nombre total de pixels dans le contour est-il un mauvais estimateur du "vrai" périmètre? **NB.** Il existe de meilleurs estimateurs mais nous allons nous contenter de celui-ci pour ce TP.
2. Compléter la fonction `bounding_box` qui calcule la boîte englobante autour d'un objet.
3. Calculer les mesures qui vous semblent pertinentes pour discriminer les objets
4. Classez les objets en générant deux images pour chaque type d'objet: oiseaux et ballons.

In [234]:
def bounding_box(image, label):
    """
    Calcule la boîte englobante d'un objet
    défini par son label

    Parameters
    ----------
    image: np.ndarray
        image
    label: int
        le label de l'objet

    Returns
    -------
    rect: np.ndarray
        le coin inférieur gauche et supérieur droit du rectangle
    """
    coords = np.argwhere(image==label)
    p_min = np.min(coords, axis=0)
    p_max = np.max(coords, axis=0)
    return [p_min, p_max]


def area(image_cc):
    return image_cc.size

def compacity(image_cc, p):
    return 4 * np.pi * area(image_cc) / p**2

def measurements(image):
    edges = morphology.dilation(image) - morphology.erosion(image)
    measures = []
    for i in range(1, image.max()+1):
        p_min, p_max = bounding_box(img_labels, i)
        p_rect = 2*(p_max[0] - p_min[0]) + 2*(p_max[1] - p_min[1])
        a = area(image[image==i])
        p = area(edges[edges==i])
        c = compacity(image[image==i], p)
        c_box = compacity(image[image==i], p_rect)
        measures.append([a, p, c, c_box])
    return measures

measures = measurements(img_labels)
balloons = [i+1 for i, m in enumerate(measures) if m[2] > 0.2]
birds = [i+1 for i, m in enumerate(measures) if m[2] <= 0.2]

zeros = np.zeros_like(img_labels)
cond_ball = np.logical_or.reduce([img_labels == b for b in balloons])
cond_bird = np.logical_or.reduce([img_labels == b for b in birds])

ball_image = np.zeros_like(img_labels)
bird_image = np.zeros_like(img_labels)
ball_image[cond_ball] = 255
bird_image[cond_bird] = 255

fig, ax = plt.subplots(1, 2)
ax[0].imshow(ball_image)
ax[1].imshow(bird_image)
[axi.set_axis_off() for axi in ax]

[None, None]

## 3. Interaction avec une vidéo et soustraction de fond

Dans cet exercice, vous allez interagir avec le flux vidéo de votre webcam, et manipuler la bibliothèque OpenCV.

L'objectif principal est d'effectuer une **suppression de fond**. On rappelle que la suppression est faite par différence avec une image statique d'où les objets mouvants sont absents. Après seuillage, on peut remplacer les pixels de fond par ceux d'une image quelconque.

<table>
    <tr>
    <td>
        <img src="data/suppression_fond_1.png" width="350" /> 
        <figcaption> Image d'arrière-plan </figcaption>
    </td>
    <td>
        <img src="data/suppression_fond_vert.png" width="350" /> 
        <figcaption> Vidéo avec soustraction de fond </figcaption>
    </td>
    </tr>
</table>

**Points pratiques**:
* Les images couleur en OpenCV sont encodées dans l'ordre **BGR**... Prudence!
* Pour quitter la vidéo, appuyer sur la touche Q.
* Pour les personnes ne disposant pas de webcam, il est possible d'utiliser la vidéo `data/extrait_voyage_occident.mp4`.


**Exercice**: Compléter le code suivant:
1. Capturer une image sans objet mouvant. Cette capture peut être effectuée par un clic ou une entrée clavier quelconque.
2. Compléter la fonction `background_subtraction` pour effectuer la suppression et le remplacement de fond. Attention au type (`uint8`) des images!
3. (Facultatif) Détecter les visages dans le flux vidéo, en vous aidant de la documentation OpenCV. Les cascades de Haar (fichiers au format `xml`) sont stockées dans le répertoire `data`.

In [298]:
import cv2
import time


def background_subtraction(frame_after, frame_before, background, threshold):
    """
    Suppression et remplacement
    de fond

    Parameters
    ----------
    frame_after: np.ndarray
        image courante du flux vidéo
    frame_before: np.ndarray
        image de référence sans objet mouvant
    background: np.ndarray
        image par laquelle remplacer les pixels statiques
    threshold: int
        seuil pour obtenir une image binaire fond/objet mouvant

    Returns
    -------
    frame: np.ndarray
        image avec suppression de fond
    """
    frame2 = np.abs((frame_after.astype(int) - frame_before.astype(int))).astype(np.uint8)
    frame2 = np.where(np.any((frame2 > threshold), axis=-1)[..., np.newaxis], frame_after, montagne).astype(np.uint8)
    return frame2

def detect_faces(frame, face_cascade):
    """
    Detection de visages
    par les cascades de Haar
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 3)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    return frame


vid = cv2.VideoCapture("data/extrait_voyage_occident.mp4")
montagne = cv2.imread("data/montagne.jpg")

haar_file = 'data/haarcascade_upperbody.xml'
face_cascade = cv2.CascadeClassifier(haar_file)
ret, first_image = vid.read()

while(vid.isOpened()):
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    if ret == False:
        break
    
    faces = detect_faces(frame, face_cascade)
    frame2 = background_subtraction(frame, first_image, montagne, 30)
    # Display the resulting frame
    cv2.imshow('frame', frame2)
    time.sleep(1/30)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

## 4. Bonus - Transformée de Hough

Nous souhaitons connaître l'équation des droites associées aux ficelles du ballon, afin d'extraire la longueur précise de chaque ficelle.

Nous allons utiliser la **transformée de Hough** pour détecter les lignes. La transformée de Hough détecte les lignes dans un espace de paramètre $(\rho, \theta)$. 

Une droite a pour équation:
\begin{equation}
    \rho = x \cos \theta + y \sin \theta
\end{equation}

Une ligne dans l'espace image devient un point dans l'espace de paramètre $(\rho, \theta)$. 

Un point dans l'espace image devient une sinusoïde dans l'espace de paramètre $(\rho, \theta)$. 

La transformée de Hough fonctionne par un système de **vote**, ou d'accumulation. On parcourt chaque pixel contour, et on trace sa sinusoïde dans $(\rho, \theta)$. A chaque passage par un pixel dans l'espace $(\rho, \theta)$, on incrémente sa valeur de 1.

<table>
    <tr>
    <td>
        <img src="data/obj_hough.png" width="700" /> 
        <figcaption style="text-align:center;"> De gauche à droite: (a) Image binaire, (b) Droites détectées par la transformée de Hough (en rouge) </figcaption>
    </td>
    </tr>
</table>

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `hough_lines` qui calcule la transformée de Hough. Cette fonction retourne l'image d'accumulation dans l'espace $(\rho, \theta)$. L'axe $\theta$ va de $0$ à $\pi$ (subdivisions en 180 intervalles), et l'axe $\rho$ de $0$ à $\sqrt{L^2 + H^2}$ où $L$ et $H$ sont la largeur et la hauteur de l'image, respectivement.
2. Filtrer l'image d'accumulation pour isoler les points majoritaires. Ces points correspondent aux lignes dans l'espace $(x, y)$.
3. Utiliser la fonction `plt.axline` pour tracer les droites. Calculer les coordonées d'un point de départ avec les équations :$x_0 = \rho * \cos \theta$ et $y_0 = \rho \sin \theta$. La pente est donnée par $\tan(-\theta)$.
4. Proposer une solution pour supprimer les droites redondantes.
5. Grâce à l'équation des droites, calculer la longueur de chaque segment.
6. (Facultatif) Comparer vos résultats avec les fonctions implémentées dans le module `skimage` pour la transformée de Hough.

In [294]:
def hough_lines(image):
    """
    Détection des lignes par la transformée de Hough
    """
    coords = np.argwhere(image > 0)
    len_diagonal = np.sqrt(image.shape[0]**2 + image.shape[1]**2)
    theta = np.linspace(0, np.pi, 180)
    theta_discrete = np.arange(theta.size).astype(int)
    rho = np.arange(len_diagonal).astype(int)
    image_hough = np.zeros((theta_discrete.size, rho.size))
    for x, y in coords:
        rho = x * np.cos(theta) + y * np.sin(theta)
        cond = (rho >= 0) & (rho < len_diagonal)
        p = rho[cond].astype(int)
        t = theta_discrete[cond].astype(int)
        image_hough[t, p] += 1
    return image_hough

def filter_hough(x_start, y_start, theta, dist):
    """
    Filtrage de la transformée de Hough pour qu'il y ait un
    espace minimal entre chaque ligne
    """
    new_x = [x_start[0]]
    new_y = [y_start[0]]
    new_theta = [theta[0]]
    for i in range(1, len(x_start)):
        x, y = x_start[i], y_start[i]
        to_add = True
        for j in range(len(new_x)):
            x2, y2 = new_x[j], new_y[j]
            if np.sqrt((x2-x)**2 + (y2-y)**2) < dist:
                to_add = False
        if to_add:
            new_x.append(x)
            new_y.append(y)
            new_theta.append(theta[i])
    return np.array(new_x), np.array(new_y), np.array(new_theta)


img_threshold2 = np.where((img_birds > 50) & (img_birds < 52), 255, 0)
hough = hough_lines(img_threshold2)
coords = np.argwhere(hough>15)
x, y = coords.T
coords = coords[np.argsort(hough[x, y])[::-1]]
linsp = np.linspace(0, np.pi, 180)
theta = coords[:, 0] * (linsp[1] - linsp[0])
rho = coords[:, 1]


x_start = rho * np.cos(theta)
y_start = rho * np.sin(theta)

x_start, y_start, theta = filter_hough(x_start, y_start, theta, 50)

image = img_threshold2.copy()

fig, ax = plt.subplots(1, 2)

ax[0].imshow(image)
for i in range(len(x_start)):
    ax[1].axline((y_start[i], x_start[i]), slope=np.tan(-theta[i]), c="r")
ax[1].imshow(image)
[axi.set_axis_off() for axi in ax]
#plt.imshow(img_threshold2)

[None, None]